In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from imblearn.over_sampling import SMOTE
import seaborn as sns
import scikitplot as skplt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV


from xgboost.sklearn import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC



from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, precision_score, f1_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset = pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
#Explore data
dataset = pd.DataFrame(dataset)


# 資料查看

In [ ]:
dataset.info()
dataset.drop(columns=['id']).describe()

從編號第8列可以看出bmi是有缺少資料的狀況

# 資料缺失

In [ ]:
#托曼·康斯坦丁（Thoman Konstantin）提供https：//www.kaggle.com/thomaskonstantin/analyzing-and-modeling-stroke-data
DT_bmi_pipe = Pipeline( steps=[ 
                               ('scale',StandardScaler()),
                               ('lr',DecisionTreeRegressor(random_state=42))
                              ])
X = dataset[['age','gender','bmi']].copy()
X.gender = X.gender.replace({'Male':0,'Female':1,'Other':-1}).astype(np.uint8)

Missing = X[X.bmi.isna()]
X = X[~X.bmi.isna()]
Y = X.pop('bmi')
DT_bmi_pipe.fit(X,Y)
predicted_bmi = pd.Series(DT_bmi_pipe.predict(Missing[['age','gender']]),index=Missing.index)
dataset.loc[Missing.index,'bmi'] = predicted_bmi
print('Missing values: ',sum(dataset.isnull().sum()))

In [ ]:
dataset.info()

In [ ]:
dataset = dataset.drop(columns=['id'])
for i in range(len(dataset.index)): #bmi
    if dataset.iloc[i, 8] < 18.5:
        dataset.iloc[i, 8] = 'Underweight'
    elif dataset.iloc[i, 8] < 24.0 and dataset.iloc[i, 8] >= 18.5:
        dataset.iloc[i, 8] = 'Normal weight'
    elif dataset.iloc[i, 8] < 30.0 and dataset.iloc[i, 8] >= 24.0:
        dataset.iloc[i, 8] = 'Overweight'
    else:
        dataset.iloc[i, 8] = 'Obese'

for i in range(len(dataset.index)):
    if dataset.iloc[i, 7] < 100.0:
        dataset.iloc[i, 7] = 'Normal'
    elif dataset.iloc[i, 7] >= 100.0 and dataset.iloc[i, 7] < 125.0:
        dataset.iloc[i, 7] = 'Prediabetes'
    else:
        dataset.iloc[i, 7] = 'Diabetes'

In [ ]:
dataset.head()

In [ ]:
le = LabelEncoder()
# dataset_fit = dataset.drop(columns=['age'])
dataset_fit = dataset.apply(le.fit_transform)
# dataset_fit.loc[:,'age'] = dataset['age']
dataset_fit.head()

In [ ]:
import plotly.express as px
corr_num = dataset_fit.corr()
# print(corr_num)
corr_matrix = pd.DataFrame(corr_num)
stroke_corr = corr_num.loc["stroke"]
print("stroke_corr \n",stroke_corr.sort_values())
px.imshow(corr_matrix)

In [ ]:
pd.DataFrame(dataset_fit)
dataset_fit.describe()

In [ ]:
str_only = dataset_fit[dataset_fit['stroke'] == 1]
no_str_only = dataset_fit[dataset_fit['stroke'] == 0]

In [ ]:
fig = plt.figure(figsize=(20,15))
gs = fig.add_gridspec(3, 2)
gs.update(wspace=0.35, hspace=0.35)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])
ax4 = fig.add_subplot(gs[2, 0])
ax5 = fig.add_subplot(gs[2, 1])

background_color = "#ffffff"
color_y = '#0d7aff' #藍中風
color_n = '#ffa230' #橘無中風
fig.patch.set_facecolor(background_color) 
ax0.set_facecolor(background_color) 
ax1.set_facecolor(background_color) 
ax2.set_facecolor(background_color)
ax3.set_facecolor(background_color) 
ax4.set_facecolor(background_color) 
ax5.set_facecolor(background_color) 


for s in ["top","right","left"]:
    ax0.spines[s].set_visible(False)
    ax1.spines[s].set_visible(False)
    ax2.spines[s].set_visible(False)
    ax3.spines[s].set_visible(False)
    ax4.spines[s].set_visible(False)
    ax5.spines[s].set_visible(False)

## Age

temp_df = pd.DataFrame(str_only["age"])
no_temp_df = pd.DataFrame(no_str_only["age"])
sns.kdeplot(temp_df["age"], ax=ax0,color=color_y, shade=True)
sns.kdeplot(no_temp_df["age"], ax=ax0, color=color_n, shade=True)
ax0.yaxis.set_major_locator(mtick.MultipleLocator(2))
ax0.set_ylabel('')    
ax0.set_xlabel('')

# heart_disease

temp_df = pd.DataFrame(str_only["heart_disease"].value_counts())
temp_df["Percentage"] = temp_df["heart_disease"].apply(lambda x: x/sum(temp_df["heart_disease"])*100)
no_temp_df = pd.DataFrame(no_str_only["heart_disease"].value_counts())
no_temp_df["Percentage"] = no_temp_df["heart_disease"].apply(lambda x: x/sum(no_temp_df["heart_disease"])*100)

x = np.arange(len(temp_df))
ax1.grid(color='gray', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
ax1.bar(x, height=temp_df["Percentage"], zorder=3, color=color_y, width=0.4)
ax1.bar(x+0.4, height=no_temp_df["Percentage"], zorder=3, color=color_n, width=0.4)
ax1.set_xticks(x + 0.4 / 2)
ax1.set_xticklabels(['No History','History'])
ax1.yaxis.set_major_formatter(mtick.PercentFormatter())
ax1.yaxis.set_major_locator(mtick.MultipleLocator(20))
for i,j in zip([0, 1], temp_df["Percentage"]):
    ax1.annotate(f'{j:0.0f}%',xy=(i, j/2), color=background_color, horizontalalignment='center', verticalalignment='center')
for i,j in zip([0, 1], no_temp_df["Percentage"]):
    ax1.annotate(f'{j:0.0f}%',xy=(i+0.4, j/2), color=background_color, horizontalalignment='center', verticalalignment='center')
    
# hypertension

temp_df = pd.DataFrame(str_only["hypertension"].value_counts())
temp_df["Percentage"] = temp_df["hypertension"].apply(lambda x: x/sum(temp_df["hypertension"])*100)

no_temp_df = pd.DataFrame(no_str_only["hypertension"].value_counts())
no_temp_df["Percentage"] = no_temp_df["hypertension"].apply(lambda x: x/sum(no_temp_df["hypertension"])*100)

x = np.arange(len(temp_df))

ax2.grid(color='gray', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
ax2.bar(x, height=temp_df["Percentage"], zorder=3, color=color_y, width=0.4)
ax2.bar(x+0.4, height=no_temp_df["Percentage"], zorder=3, color=color_n, width=0.4)
ax2.set_xticks(x + 0.4 / 2)
ax2.set_xticklabels(['No History','History'])
ax2.yaxis.set_major_formatter(mtick.PercentFormatter())
ax2.yaxis.set_major_locator(mtick.MultipleLocator(20))
for i,j in zip([0, 1], temp_df["Percentage"]):
    ax2.annotate(f'{j:0.0f}%',xy=(i, j/2), color=background_color, horizontalalignment='center', verticalalignment='center')

for i,j in zip([0, 1], no_temp_df["Percentage"]):
    ax2.annotate(f'{j:0.0f}%',xy=(i+0.4, j/2), color=background_color, horizontalalignment='center', verticalalignment='center')

# ever_married
temp_df = pd.DataFrame(str_only["ever_married"].value_counts())
temp_df["Percentage"] = temp_df["ever_married"].apply(lambda x: x/sum(temp_df["ever_married"])*100)

no_temp_df = pd.DataFrame(no_str_only["ever_married"].value_counts())
no_temp_df["Percentage"] = no_temp_df["ever_married"].apply(lambda x: x/sum(no_temp_df["ever_married"])*100)

x = np.arange(len(temp_df))


ax3.grid(color='gray', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
ax3.bar(x, height=temp_df["Percentage"], zorder=3, color=color_y, width=0.4)
ax3.bar(x+0.4, height=no_temp_df["Percentage"], zorder=3, color=color_n, width=0.4)
ax3.set_xticks(x + 0.4 / 2)
ax3.set_xticklabels(['No Married','Married'])
ax3.yaxis.set_major_formatter(mtick.PercentFormatter())
ax3.yaxis.set_major_locator(mtick.MultipleLocator(10))
for i,j in zip([0, 1], temp_df["Percentage"]):
    ax3.annotate(f'{j:0.0f}%',xy=(i, j/2), color=background_color, horizontalalignment='center', verticalalignment='center')
for i,j in zip([0, 1], no_temp_df["Percentage"]):
    ax3.annotate(f'{j:0.0f}%',xy=(i+0.4, j/2), color=background_color, horizontalalignment='center', verticalalignment='center')


# smoking_status
temp_df = pd.DataFrame(str_only["smoking_status"].value_counts())
temp_df["Percentage"] = temp_df["smoking_status"].apply(lambda x: x/sum(temp_df["smoking_status"])*100)
no_temp_df = pd.DataFrame(no_str_only["smoking_status"].value_counts())
no_temp_df["Percentage"] = no_temp_df["smoking_status"].apply(lambda x: x/sum(no_temp_df["smoking_status"])*100)


ax4.barh(temp_df.index, temp_df['Percentage'], color=color_y, zorder=3, height=0.7)
ax4.barh(no_temp_df.index, no_temp_df['Percentage'], color=color_n, zorder=3, height=0.3)
ax4.xaxis.set_major_formatter(mtick.PercentFormatter())
ax4.xaxis.set_major_locator(mtick.MultipleLocator(10))


# bmi

temp_df = pd.DataFrame(str_only["bmi"].value_counts())
temp_df["Percentage"] = temp_df["bmi"].apply(lambda x: x/sum(temp_df["bmi"])*100)
no_temp_df = pd.DataFrame(no_str_only["bmi"].value_counts())
no_temp_df["Percentage"] = no_temp_df["bmi"].apply(lambda x: x/sum(no_temp_df["bmi"])*100)


ax5.barh(temp_df.index, temp_df['Percentage'], color=color_y, zorder=3, height=0.7)
ax5.barh(no_temp_df.index, no_temp_df['Percentage'], color=color_n, zorder=3, height=0.3)
ax5.xaxis.set_major_formatter(mtick.PercentFormatter())
ax5.xaxis.set_major_locator(mtick.MultipleLocator(10))



#text

font_size = 16
font_weight = 'bold'
font_family =  'Arial'
ax0.text(-22, 0.041, 'Age', fontsize=font_size ,fontweight=font_weight,fontfamily=font_family)

ax1.text(-0.4, 110, 'Heart Disease', fontsize=font_size ,fontweight=font_weight,fontfamily=font_family)

ax2.text(-0.4, 100, 'Hypertension', fontsize=font_size ,fontweight=font_weight,fontfamily=font_family)

ax3.text(-0.4, 100, 'Ever Married', fontsize=font_size ,fontweight=font_weight,fontfamily=font_family)


ax4.text(-3, 3.5, 'Smoking Status', fontsize=font_size ,fontweight=font_weight,fontfamily=font_family)


ax5.text(-3, 3.5, 'bmi', fontsize=font_size ,fontweight=font_weight,fontfamily=font_family)



plt.show()

In [ ]:
dataset_fit['stroke'].value_counts()



# 訓練集分割

In [ ]:
X  = dataset_fit[['age','heart_disease','hypertension','ever_married','smoking_status','bmi']]
y = dataset_fit['stroke']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

sm = SMOTE()
X_train_resh, y_train_resh = sm.fit_resample(X_train, y_train.ravel())
X_test_resh, y_test_resh = sm.fit_resample(X_test, y_test.ravel())

# 訓練

In [ ]:
svm_pipeline = Pipeline(steps = [('SVM',SVC(random_state=42))])

rf_pipeline = Pipeline(steps = [('RF',RandomForestClassifier(random_state=42))])

DT_pipeline = Pipeline(steps = [('DT',DecisionTreeClassifier(random_state=42))])

XGB_pipeline = Pipeline(steps = [('XGB',XGBClassifier(random_state=42))])

交叉驗證

In [ ]:
svm_pipeline_mean = cross_val_score(svm_pipeline,X_train_resh,y_train_resh,scoring='f1').mean()

rf_pipeline_mean = cross_val_score(rf_pipeline,X_train_resh,y_train_resh,scoring='f1').mean()

DT_pipeline_mean = cross_val_score(DT_pipeline,X_train_resh,y_train_resh,scoring='f1').mean()

XGB_pipeline_mean = cross_val_score(XGB_pipeline,X_train_resh,y_train_resh,scoring='f1').mean()

交叉驗證10次

In [ ]:
svm_pipeline_mean_cv = cross_val_score(svm_pipeline,X_train_resh,y_train_resh,cv=10,scoring='f1').mean()

rf_pipeline_mean_cv = cross_val_score(rf_pipeline,X_train_resh,y_train_resh,cv=10,scoring='f1').mean()

DT_pipeline_mean_cv = cross_val_score(DT_pipeline,X_train_resh,y_train_resh,cv=10,scoring='f1').mean()

XGB_pipeline_mean_cv = cross_val_score(XGB_pipeline,X_train_resh,y_train_resh,cv=10,scoring='f1').mean()

In [ ]:
print('Mean f1 scores:')
print('SVM mean :',svm_pipeline_mean)
print('Random Forest mean :',rf_pipeline_mean)
print('DecisionTreeClassifier mean :',DT_pipeline_mean)
print('XGBoost mean :',XGB_pipeline_mean)

In [ ]:
print('Mean f1 scores(After cv = 10):')
print('SVM mean :',svm_pipeline_mean_cv)
print('Random Forest mean :',rf_pipeline_mean_cv)
print('DecisionTreeClassifier mean :',DT_pipeline_mean_cv)
print('XGBoost mean :',XGB_pipeline_mean_cv)

# Random Forest

找尋最佳參數

In [ ]:
n_estimators =[64,100,128]
max_features = ['auto',2,3,5,7]
bootstrap = [True,False]

param_grid = {'n_estimators':n_estimators,
             'max_features':max_features,
             'bootstrap':bootstrap}
rfc = RandomForestClassifier()
grid = GridSearchCV(rfc,param_grid)

grid.fit(X_train_resh, y_train_resh)
grid.best_params_

In [ ]:
rf4 = RandomForestClassifier(bootstrap = True, max_features = 'auto', n_estimators = 128)
rf4.fit(X_train_resh, y_train_resh)
predprob4 = rf4.predict(X_test)
rf4_f10  = f1_score(y_test,predprob4)
print('RF mean :',rf4_f10)
from sklearn.metrics import classification_report
print(classification_report(y_test,predprob4))